In [ ]:
import re
from pathlib import Path
import pandas as pd
import pdb

In [ ]:
data_folder = Path("/Users/nicolasroever/Dropbox/Promotion/LVT/landvaluetax/src/landvaluetax/data/raw_data_res_land")

In [ ]:
all_dfs = []

In [ ]:
for file_path in data_folder.glob("*.xlsx"):
    # --- a) Read the transaction_type (first row, first cell) ---
    transaction_type = pd.read_excel(file_path, header=None, nrows=1).iloc[0, 0]

    # --- b) Read the info string (second row, first cell) ---
    info_str = pd.read_excel(file_path, header=None, skiprows=1, nrows=1).iloc[0, 0]
    # e.g. "Harju maakond, Anija vald on time period 9/1/2021 and 9/30/2021"
    m = re.match(
        r"^(?P<county>[^,]+),\s*(?P<municipality>.+?)\s+on time period\s+"
        r"(?P<start>\d{1,2}/\d{1,2}/\d{4})\s+and\s+(?P<end>\d{1,2}/\d{1,2}/\d{4})",
        info_str,
    )
    if not m:
        raise ValueError(f"Unrecognized info format in {file_path}: {info_str!r}")

    county = m.group("county")
    municipality = m.group("municipality")
    # extract month and year from the start date
    start_month, _, start_year = m.group("start").split("/")

    # --- c) Read the data table starting at row 7 (skip 6 rows) ---
    raw = pd.read_excel(file_path, header=None, skiprows=6)
    # drop everything at and after the first blank row
    blank_idx = raw[raw.isna().all(axis=1)].index
    if len(blank_idx):
        raw = raw.loc[: blank_idx[0] - 1]

    # assign your column names
    raw.columns = [
        'house_type',
        "number_trans",
        "average_sqm",
        "total_price",
        "min_price",
        "max_price",
        "min_price_per_sqm",
        "max_price_per_sqm",
        "median_price_per_sqm",
        "average_price_per_sqm",
        "sd_price_per_sqm",
    ]

    # --- d) Add the metadata columns ---
    raw["transaction_type"] = transaction_type
    raw["county"] = county
    raw["municipality"] = municipality
    raw["month"] = int(start_month)
    raw["year"] = int(start_year)

    # collect
    all_dfs.append(raw)

# 4) Concatenate all into one big DataFrame
result = pd.concat(all_dfs, ignore_index=True)

In [ ]:
result.head()